# Example
of the last added functionalities in onlineSG

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import sys
sys.path.append('../')  # uncomment to add the path to the onlineSG folder 
# You have to change the path if you move this file.
# Alternatevely add the onlineSG folder to your PYTHONPATH
# e.g. On Linux add this line on .bashrc:
# export PYTHONPATH="<absolute-path-to-onlineSG>:PYTHONPATH"
# then you can use onlineSG library from everywhere

In [1]:
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.parsers as parsers
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.runner as runner
import source.players.detection as detection 
import source.util as util

from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time

In [94]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(runner)
reload(parsers)
reload(game)
#reload(environment)
reload(detection)
reload(util)

<module 'source.util' from '/home/lorenzo/Scrivania/Polimi/Thesis/code/onlineSG/source/util.py'>

In [2]:
targets_dict = {i:[] for i in range(2,13)}
T = 15 #np.random.randint(2,10)
time_horizon =1000
while sum([len(targets_dict[x]) for x in range(2,13)]) < 9:
    targets = [round(x,3) for x in util.gen_norm_targets(T)]
    values = tuple((v, v) for v in targets)
    g = game.Game(values, time_horizon)
    g.attackers = [1]
    g.defenders = [0]
    s = util.support(g)
    if len(targets_dict[len(s)])==0:
        targets_dict[len(s)].append(s)

In [98]:
T = 10
P = 2
targets = translate(targets_dict[T][0])
distributions = []
for i in range(P):
    distributions.append(tuple(gen_distr(T)))
values = tuple((v, v) for v in targets)
g = game.Game(values, time_horizon)
#print(T, targets)
profiles = [attackers.StackelbergAttacker(g, 1)]
for d in distributions:
    #print(d)
    profiles.append(attackers.StochasticAttacker(g, 1, 1, *d))
agent = detection.HOLMES(g, 0, 1, 1)
attacker = profiles[0]
g.set_players([agent], [attacker], [p for p in profiles if p != attacker])

In [33]:
T = 6
P * P * T + T * T * T * P

456

In [34]:
1 * (456 / 33)

13.818181818181818

In [99]:
e = runner.Experiment(g)

In [101]:
start_time = time.time()
e.run_interaction()
print(time.time() - start_time)
print(game_str(e.game, lenght=7))
print(e.agent.belief)
print(e.game.strategy_history[-1])


5.467822313308716
 	 	 	 	□	 	 	 	 	 	
0.946	0.841	0.948	0.924	0.939	0.968	0.875	0.829	0.942	0.949	
 	 	 	 	 	△	 	 	 	 	
----------------------------------------------------------------------
□	 	 	 	 	 	 	 	 	 	
0.946	0.841	0.948	0.924	0.939	0.968	0.875	0.829	0.942	0.949	
 	 	 	 	 	 	 	△	 	 	
{<StochasticAttacker id:1 resources:1>: 4.8584924907380388e-05, <StackelbergAttacker id:1 resources:1>: 0.96330742561762162, <StochasticAttacker id:1 resources:1>: 0.036643989457471077}
{0: (0.13083354170947906, 0.022315789674063023, 0.13266724533539395, 0.1101388865027252, 0.12435406499937716, 0.15058753077047146, 0.06030615676486166, 0.008163414223364602, 0.12705217124434545, 0.13358119877591845), 1: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]}


In [88]:
from copy import deepcopy

{<StochasticAttacker id:1 resources:1>: <KnownStochasticDefender id:0 resources:1>,
 <StochasticAttacker id:1 resources:1>: <KnownStochasticDefender id:0 resources:1>,
 <StackelbergAttacker id:1 resources:1>: <StackelbergDefender id:0 resources:1>}

In [93]:
for k in e.agent.profiles:
    start_time = time.time()
    k.get_best_responder()
    print(time.time() - start_time)

2.8848648071289062e-05
4.291534423828125e-05
3.552436828613281e-05


In [83]:
start_time = time.time()
e.run_interaction()
print(time.time() - start_time)
print(game_str(e.game, lenght=7))
print(e.agent.belief)
print(e.game.strategy_history[-1])


3 0.1614851951599121
2 0.12384796142578125
2 0.13333487510681152
4 0.11460113525390625
4 0.1505756378173828
5 0.11072969436645508
5 0.2053072452545166
5 0.12882423400878906
0 0.10135197639465332
0 0.14489126205444336
0 0.11942005157470703
0 0.11493778228759766
0 0.10756826400756836
0 0.10907673835754395
1 0.12592172622680664
1 0.1139230728149414
1 0.11368942260742188
1 0.1501328945159912
1 0.20796728134155273
1 0.1350266933441162
2 0.1182558536529541
2 0.14227032661437988
2 0.13663363456726074
2 0.12044191360473633
2 0.135117769241333
2 0.13111209869384766
3 0.12725615501403809
3 0.14376473426818848
3 0.19898462295532227
3 0.22863268852233887
3 0.1897735595703125
3 0.20658540725708008
4 0.14933109283447266
4 0.18760180473327637
4 0.1394650936126709
4 0.15960359573364258
4 0.1792283058166504
4 0.14325928688049316
5 0.14247345924377441
5 0.15726852416992188
5 0.24408602714538574
5 0.23748445510864258
5 0.1849346160888672
5 0.19228911399841309
0 0.21271991729736328
0 0.15095949172973633
0

In [77]:
start_time = time.time()
e.run_interaction()
print(time.time() - start_time)
print(game_str(e.game, lenght=7))
print(e.agent.belief)
print(e.game.strategy_history[-1])


5 0.10023260116577148
0 0.12727952003479004
0 0.1606607437133789
0 0.12775063514709473
1 0.11857318878173828
1 0.11449837684631348
2 0.12177371978759766
3 0.10972452163696289
3 0.10124850273132324
4 0.1665353775024414
4 0.10376882553100586
4 0.21407008171081543
4 0.12234997749328613
5 0.10926699638366699
5 0.12927746772766113
5 0.17539024353027344
5 0.10990071296691895
5 0.11029887199401855
5 0.17348337173461914
0 0.1103827953338623
0 0.11120414733886719
0 0.13007712364196777
0 0.257218599319458
0 0.12633252143859863
0 0.11751651763916016
1 0.18093466758728027
1 0.11615347862243652
1 0.11476969718933105
1 0.12985491752624512
1 0.17724108695983887
1 0.21025371551513672
2 0.2185661792755127
2 0.13093066215515137
2 0.13569355010986328
2 0.24854636192321777
2 0.13298654556274414
2 0.13535690307617188
3 0.14589619636535645
3 0.14407825469970703
3 0.1370389461517334
3 0.14382481575012207
3 0.15672636032104492
3 0.14872217178344727
4 0.14051246643066406
4 0.1767728328704834
4 0.21783638000488

In [73]:
e.agent.get_t_strategies()

[(-0.0003953316692775887,
  0.07925975740110955,
  0.18881587316117843,
  0.18881587316117837,
  0.12738829523450437,
  0.41611553271130686),
 (0.011604668330722412,
  0.06725975740110955,
  0.18881587316117843,
  0.18881587316117837,
  0.12738829523450437,
  0.41611553271130686),
 (0.011604668330722412,
  0.07925975740110955,
  0.17681587316117842,
  0.18881587316117837,
  0.12738829523450437,
  0.41611553271130686),
 (0.011604668330722412,
  0.07925975740110955,
  0.18881587316117843,
  0.17681587316117836,
  0.12738829523450437,
  0.41611553271130686),
 (0.011604668330722412,
  0.07925975740110955,
  0.18881587316117843,
  0.18881587316117837,
  0.11538829523450438,
  0.41611553271130686),
 (0.011604668330722412,
  0.07925975740110955,
  0.18881587316117843,
  0.18881587316117837,
  0.12738829523450437,
  0.40411553271130685)]

In [67]:
e.game.profiles[1].get_best_responder().compute_strategy()

[0, 0, 0, 0, 1, 0]

In [68]:
e.game.profiles[2].get_best_responder().compute_strategy()

[0, 0, 0, 0, 0, 1]

In [15]:
e.agent.profiles[0].game.strategy_history

[{0: (0.28555077452667826, 0.48176419965576583, 0.23268502581755596),
  1: [0, 0, 1]},
 {0: (0.28555077452667826, 0.48176419965576583, 0.23268502581755596),
  1: [0, 0, 1]}]

In [16]:
e.game.strategy_history

[{0: (0.28555077452667826, 0.48176419965576583, 0.23268502581755596),
  1: [0, 0, 1]},
 {0: (0.28555077452667826, 0.48176419965576583, 0.23268502581755596),
  1: [0, 0, 1]}]

In [89]:
e.run_interaction()

In [79]:
e.game.players[1].game.strategy_history[-1]

{0: (0.3690684156632389, 0.4282480801547355, 0.20268350418202563),
 1: [0, 0, 1]}

In [80]:
e.game.strategy_history[-1]

{0: (0.3690684156632389, 0.4282480801547355, 0.20268350418202563),
 1: [0, 0, 1]}

In [78]:
e.agent.profiles[0].game.strategy_history[-1]

{0: (0, 1, 0), 1: [0, 0, 1]}

In [65]:
e.game.strategy_history[-1] 

{0: (0.07000285365649891,
  0.30581066825329867,
  0.14416530551802176,
  0.3511386294556226,
  0.1288825431165581),
 1: [1, 0, 0, 0, 0]}

In [37]:
start_time = time.time()
e.run()
print(time.time() - start_time)

378.01570677757263


In [ ]:
start_time = time.time()

In [24]:
time_horizon = 100
targets = (1,2,3)
values = tuple((v, v) for v in targets)
distribution1 = (0,0.5,0.5)
distribution2 = (0,0.1,0.9)
g = game.Game(values, time_horizon)
#profiles = [attackers.StackelbergAttacker(g, 1, 1)] #,
profiles = [attackers.StochasticAttacker(g, 1, 1, *distribution1)]#, attackers.StochasticAttacker(g, 1, 1, *distribution2)] #,
agent = detection.StrategyAwareDetector(g, 0, 1)
attacker = attackers.StackelbergAttacker(g, 1, 1)
#attacker = attackers.StochasticAttacker(g, 1, 1, *distribution1)
g.set_players([agent], [attacker], profiles)

In [26]:
c = runner.Configuration(g, "concurrent", "prova", False)

In [25]:
rm -rf concurrent

In [27]:
c.run(3)

In [724]:
e = runner.Experiment(g)

In [756]:
e.run_interaction()
#print(game_str(e.game, lenght=7))
print(e.agent.belief, e.agent.detection)
print(e.game.strategy_history[-1])

[0, 0.9, 0.1]
0.0
False
0.5
False
0.5
False
{<StochasticAttacker id:1 resources:1>: 1.853020188851843e-17, <StackelbergAttacker id:1 resources:1>: 1.0} None
{0: [0.0, 0.39, 0.61], 1: [0, 1, 0]}


In [757]:
e.agent.tau

32

In [720]:
e.run()

[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
False
0.0
False
[0, 0.01, 0.99]
0.0
False
1.0
Fals

In [74]:
time_horizon = 100
T = 4
targets = (5,4,8,1)
values = tuple((v, v) for v in targets)
distribution1 = (0.17,0.8, 0.01, 0.02)
g = game.Game(values, time_horizon)

other_kinds = [attackers.StochasticAttacker(g, 1, 1, *distribution1)] #,
strategy_aware = attackers.StackelbergAttacker(g, 1, 1)
L = 2
agent = detection.StrategyAwareDetector.parse("sad1-0.01@stackelberg@stochastic_attacker1-0.17-0.8-0.01-0.02", g, 0) 
#agent = detection.HOLMES(g, 0, 1, strategy_aware, other_kinds, L)
# attacker = attackers.StackelbergAttacker(g, 1, 1)
attacker = attackers.StochasticAttacker(g, 1, 1, *distribution1)
g.set_players([agent], [attacker])

In [76]:
e1 = runner.Experiment(g)

In [77]:
e1.run_interaction()
print(game_str(e1.game, lenght=7))
print(e1.agent.belief)
print(e1.game.strategy_history[-1])

□	 	 	 	
5	4	8	1	
 	△	 	 	
{<StackelbergAttacker id:1 resources:1>: 0.0, <StochasticAttacker id:1 resources:1>: 1.0}
{0: [0.30934782608695643, 0.13543478260869565, 0.5552173913043479, 0.0], 1: [0.17, 0.8, 0.01, 0.02]}
